In [1]:
!nvidia-smi

Fri Jan 31 12:36:36 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   51C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import os
os.environ["HF_HOME"] = "/openbayes/home/huggingface"

In [3]:
!echo $HF_HOME

/openbayes/home/huggingface


In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer


# 加载模型和 tokenizer （第一次加载的时候会进行下载）
model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2.5-0.5B-Instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
# 查看 模型结构是什么？
print(model)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear(in_features=896, out_features=896, bias=True)
          (k_proj): Linear(in_features=896, out_features=128, bias=True)
          (v_proj): Linear(in_features=896, out_features=128, bias=True)
          (o_proj): Linear(in_features=896, out_features=896, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((

In [5]:
# 这里有一个 Special token, 打印看一下是什么token呢？
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-0.5B-Instruct")
# tokenizer.all_special_tokens
tokenizer.special_tokens_map

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

{'eos_token': '<|im_end|>',
 'pad_token': '<|endoftext|>',
 'additional_special_tokens': ['<|im_start|>',
  '<|im_end|>',
  '<|object_ref_start|>',
  '<|object_ref_end|>',
  '<|box_start|>',
  '<|box_end|>',
  '<|quad_start|>',
  '<|quad_end|>',
  '<|vision_start|>',
  '<|vision_end|>',
  '<|vision_pad|>',
  '<|image_pad|>',
  '<|video_pad|>']}

In [7]:
from transformers import Qwen2Tokenizer
tokenizer1 = Qwen2Tokenizer.from_pretrained("Qwen/Qwen2.5-0.5B-Instruct")
tokenizer1.special_tokens_map   # 这样和autotokenizer出来的tokenizer是一样的

{'eos_token': '<|im_end|>',
 'pad_token': '<|endoftext|>',
 'additional_special_tokens': ['<|im_start|>',
  '<|im_end|>',
  '<|object_ref_start|>',
  '<|object_ref_end|>',
  '<|box_start|>',
  '<|box_end|>',
  '<|quad_start|>',
  '<|quad_end|>',
  '<|vision_start|>',
  '<|vision_end|>',
  '<|vision_pad|>',
  '<|image_pad|>',
  '<|video_pad|>']}

In [8]:
prompt = '说一下大模型的研究方向'

messages = [{'role':'system','content':'you are qwen, a helpful assistant'},
       {'role':'user','content': prompt}
       ]

In [11]:
text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
text

'<|im_start|>system\nyou are qwen, a helpful assistant<|im_end|>\n<|im_start|>user\n说一下大模型的研究方向<|im_end|>\n'

In [12]:
text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
text  # 就是多了一个assistant  # 加入特殊符号以及后续生成的开头

'<|im_start|>system\nyou are qwen, a helpful assistant<|im_end|>\n<|im_start|>user\n说一下大模型的研究方向<|im_end|>\n<|im_start|>assistant\n'

In [13]:
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

print(text)
print("====" * 10)
print(model_inputs)

<|im_start|>system
you are qwen, a helpful assistant<|im_end|>
<|im_start|>user
说一下大模型的研究方向<|im_end|>
<|im_start|>assistant

{'input_ids': tensor([[151644,   8948,    198,   9330,    525,   2804,  16948,     11,    264,
          10950,  17847, 151645,    198, 151644,    872,    198,  36587, 100158,
          26288, 104949, 105647, 100696, 151645,    198, 151644,  77091,    198]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1]], device='cuda:0')}


In [14]:
# step3: 推理和解码
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]  # generate_ids中仍然保存着前序prompt 这里通过列表表达式去除

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(response)

作为大模型，我的研究主要集中在以下几个方向：

1. **深度学习和神经网络**：大模型依赖于深度学习的原理和技术，因此对神经网络结构、参数优化、数据预处理等方面有着深入的理解。

2. **生成对抗网络（GAN）与强化学习**：GANs在图像生成、视频生成等领域有广泛的应用。而强化学习是大模型中一个重要的领域，它能够模拟人类的学习行为，应用于机器人、自动驾驶等场景。

3. **元语言理解和生成**：大模型可以理解并生成自然语言文本，这对于自然语言处理和对话系统的发展具有重要意义。这涉及到多模态理解、情感分析等多个方面。

4. **大规模计算和分布式训练**：随着大数据时代的到来，我们需要高效地处理海量数据，并通过分布式计算来实现高效的训练过程。这要求我们开发出高性能的大规模计算框架和算法。

5. **安全性和隐私保护**：随着人工智能技术的广泛应用，如何确保数据的安全性和用户的隐私成为了一个重要课题。这包括数据加密、匿名化处理以及隐私保护的技术研究。

6. **跨学科交叉融合**：大模型不仅限于单个领域的知识，而是需要与其他学科如计算机科学、心理学、社会学等进行交叉融合，以更全面地理解复杂的人工智能问题。

7. **伦理和法律问题**：随着人工智能技术的快速发展，相关的伦理和法律问题也日益受到关注。如何在保证技术创新的同时，避免潜在的社会风险是一个亟待解决的问题。

以上就是我关于大模型的一些研究方向概述，希望对你有所帮助。如果你有任何具体的研究项目或主题感兴趣，请告诉我，我会尽力提供更多的信息和支持。


In [17]:
generated_ids

[tensor([100622,  26288, 104949,   3837,  97611,  99556,  99558, 110678, 116420,
         100696,  48443,     16,     13,   3070, 102217, 100134,  33108, 102398,
          71356,    334,   5122,  26288, 104949, 105537,  34204, 102217, 100134,
           9370, 105318, 108800,   3837, 101886,  32664, 102398,  71356, 100166,
           5373,  32665, 103983,   5373,  20074,  98841,  54542, 102159, 104012,
         100403, 108894,   3407,     17,     13,   3070,  43959, 107126,  71356,
           9909,  58487,   7552,  57218, 103980, 100134,    334,   5122,  58487,
             82,  18493, 107553,  43959,   5373,  87140,  43959, 106483,  18830,
         100789, 106736,   1773,  68536, 103980, 100134,  20412,  26288, 104949,
          15946,  46944, 100799, 100650,   3837,  99652, 100006, 105717, 103971,
         105595, 101070,   3837, 110645, 104354,   5373, 109044,  49567, 102122,
           3407,     18,     13,   3070,  23305, 102064, 115167,  43959,    334,
           5122,  26288, 104

In [18]:
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(response)

system
you are qwen, a helpful assistant
user
说一下大模型的研究方向
assistant
作为人工智能领域的语言模型，我专注于理解和回答问题、创造文本和生成自然语言。我的研究领域主要集中在以下几方面：

1. **多模态学习与综合推理**：大模型需要处理不同类型的输入数据（如文本、图像、音频等），并能够进行跨模态的推理和综合分析。

2. **深度强化学习与自主决策**：大模型在面对复杂的任务时，需要具备自主决策的能力，能够在不同的环境下做出最佳选择。

3. **对话系统设计**：如何构建一个能够高效且自然地进行对话的大模型，包括理解用户意图、生成合适的回复以及实现自然流畅的交流。

4. **医疗健康应用**：大模型可以用于疾病的早期诊断、药物研发、患者管理等领域，为医疗服务提供技术支持。

5. **自动驾驶与机器人技术**：通过大模型，我们有望开发出更智能、更安全的自动驾驶汽车和机器人，解决交通拥堵、事故预防等问题。

6. **教育和培训**：大模型可以帮助教师更好地教授知识，提供个性化学习路径，并促进在线教育的发展。

7. **环境保护与可持续发展**：利用大数据和AI技术，大模型可以预测气候变化趋势，帮助制定有效的环境政策，推动绿色能源的发展。

8. **法律和合规性审查**：大模型可以通过分析大量司法判决、行政决定和新闻报道中的信息，协助识别潜在的法律漏洞或不公正行为。

9. **艺术创作与内容生成**：大模型可以在艺术创作中发挥重要作用，通过生成高质量的艺术作品来满足人类审美需求。

这些只是大模型研究的一部分方向，实际上大模型的应用场景非常广泛，涵盖了从科学研究到日常生活各个层面。随着技术的进步，大模型将继续扩展其功能和应用场景，对我们的生活和社会产生深远影响。


# 使用pipeline简化流程

In [15]:
from transformers import pipeline


# step1: 生成 pipeline
# return_full_text=False 表示只返回新生成的文本，不包含输入的prompt
# return_full_text=True 则会返回完整文本，包含输入的prompt和生成的新文本
generator = pipeline(
    "text-generation", # 一般都是text-generation（decoderonly），也可以是text2text-generation（encoder-decoder）
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,  # 只返回新生成的文本部分
    max_new_tokens=500,
    do_sample=False
)

# step2: 构建 prompt
messages = [
    {"role": "user", "content": "说一下大模型的研究方向."}
]

# step3，输出并解码
output = generator(messages)
print(output[0]["generated_text"])

Device set to use cuda
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:650: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


作为阿里云开发的超大规模语言模型，我专注于研究和探索如何更好地理解和生成人类语言。我的目标是使用户能够更自然、高效地表达自己的想法，并提供准确、有用的信息。

在研究方向上，我会关注以下几个方面：

1. **多模态学习**：通过结合视觉、听觉等多种形式的数据，提高模型对复杂场景的理解能力。
2. **语义理解与推理**：深入研究如何让模型理解文本中的上下文信息，进行逻辑推理和问题解决。
3. **对话系统设计**：优化对话流程，提升用户体验，实现更加流畅和自然的交流。
4. **个性化推荐**：基于用户的兴趣和行为数据，为用户提供个性化的建议和服务。
5. **跨语言处理**：支持多种语言之间的无缝转换和交互，满足全球用户的需求。
6. **安全性和隐私保护**：确保模型不会泄露敏感信息，同时尊重用户隐私。

此外，我还致力于推动AI技术的发展，包括但不限于人工智能算法的创新应用、伦理规范的制定等，以期在未来的大模型时代中发挥更大的作用。


In [16]:
output

[{'generated_text': '作为阿里云开发的超大规模语言模型，我专注于研究和探索如何更好地理解和生成人类语言。我的目标是使用户能够更自然、高效地表达自己的想法，并提供准确、有用的信息。\n\n在研究方向上，我会关注以下几个方面：\n\n1. **多模态学习**：通过结合视觉、听觉等多种形式的数据，提高模型对复杂场景的理解能力。\n2. **语义理解与推理**：深入研究如何让模型理解文本中的上下文信息，进行逻辑推理和问题解决。\n3. **对话系统设计**：优化对话流程，提升用户体验，实现更加流畅和自然的交流。\n4. **个性化推荐**：基于用户的兴趣和行为数据，为用户提供个性化的建议和服务。\n5. **跨语言处理**：支持多种语言之间的无缝转换和交互，满足全球用户的需求。\n6. **安全性和隐私保护**：确保模型不会泄露敏感信息，同时尊重用户隐私。\n\n此外，我还致力于推动AI技术的发展，包括但不限于人工智能算法的创新应用、伦理规范的制定等，以期在未来的大模型时代中发挥更大的作用。'}]